In [6]:
import random
from copy import deepcopy
from time import sleep
import time
from plotly import graph_objects as go
import ipywidgets as ipw
from IPython import display

In [7]:
class Amazons:
    """
    Core game engine responsible for managing game state, visualization, and player interactions.
    The game revolves around two amazons (P1 and P2) attempting to outmaneuver each other on a grid.
    """
    def __init__(self, state, player_1, player_2, time_to_play=10):
        self.N = len(state)
        self.time_to_play = time_to_play
        
        # Deep copy the initial state to prevent unintended modifications
        self.state_original = state
        self.state = deepcopy(state)
        
        # Assign players with symbols
        self.player_1 = {**player_1, 'symbol': 'P1'}
        self.player_2 = {**player_2, 'symbol': 'P2'}
        self.next_to_play = self.player_1
        
        # Define visual representations
        self.symbols_fig = {
            'P1': 'star',          # Player 1 amazon
            'P2': 'square',        # Player 2 amazon
            'x': 'x',             # Blocked square
            '·': 'circle-open'    # Empty square
        }
        self.colors = {
            'background': '#F5F5F5',
            'grid': '#E0E0E0',
            'P1_fill': '#FFD700',  
            'P1_line': '#FFA500',
            'P2_fill': '#4169E1',  
            'P2_line': '#1E90FF',
            'x_fill': 'rgba(255, 68, 68, 0.9)',
            'x_line': 'rgba(204, 0, 0, 1)',
            'text': '#333333',
            'button': '#4CAF50'
        }
        
        # Define scoring system
        self.scores = {'P1': 1, 'P2': -1}
        
        # Initialize UI components
        self.create_ui()
        
        # Set up layout
        self.setup_display()
        
        # Start game
        self.restart()
    
    def setup_display(self):
        hbox = ipw.HBox([self.bt_restart, self.dashboard])
        self.output = ipw.Output()
        vbox = ipw.VBox([hbox, self.fig, self.output])
        display.display(vbox)

    def restart(self, *args):
        """"Resets the game to its initial configuration."""
        self.reset_state()
        self.reset_ui()
        self.reset_board()

    def reset_state(self):
        """Resets all game state variables to their initial values."""
        self.h_calls = 0
        self.max_depth = 0
        self.next_to_play = self.player_1
        self.next_human_move = 'amazon_moves'
        self.winner = 'keep_playing'
        self.state = deepcopy(self.state_original)
        self.evaluated = {}
    
    def reset_ui(self):
        """Updates UI elements to reflect the game's initial state."""
        self.update_score('На ред е', self.next_to_play)
        if self.next_to_play['type'] == 'human':
            self.dashboard.value += ' Помести ја амазонката!'
        self.output.clear_output()
    
    def reset_board(self):
        """Refreshes the visual representation of the game board."""
        self.fig.data[0].marker.symbol = self.map_state_to_visual_symbols()

    def create_ui(self):
        self.create_dashboard()
        self.create_restart_button()
        self.fig = self.create_fig()
    
    def create_dashboard(self):
        self.dashboard = ipw.HTML(
            description='Статус:',
            value='',
            style={'description_width': 'initial'},
            layout=ipw.Layout(width='auto', margin='10px 0')
        )
    
    def create_restart_button(self):
        self.bt_restart = ipw.Button(
            description='↻ Нова игра',
            style=ipw.ButtonStyle(
                button_color=self.colors['button'],
                font_weight='bold'
            ),
            layout=ipw.Layout(width='150px', height='40px', margin='0 10px')
        )
        self.bt_restart.on_click(self.handle_restart_click)
        
    def create_fig(self):
        """Generate the game board with pieces centered at their positions."""
        fig = go.FigureWidget()
        
        # Compute positions for the markers on the grid
        x = [x + 0.5 for y in range(self.N) for x in range(self.N)]
        y = [y + 0.5 for y in range(self.N) for x in range(self.N)]
        
        # Map the current game state to the corresponding symbols for each marker
        symbols = [self.symbols_fig[v] for row in self.state for v in row]
        
        # Define colors for the fill of the markers based on their type (star, square, x)
        colors = [
            self.colors['P1_fill'] if s == 'star' else 
            self.colors['P2_fill'] if s == 'square' else 
            self.colors['x_fill'] 
            for s in symbols
        ]
        
        # Define colors for the borders of the markers based on their type
        line_colors = [
            self.colors['P1_line'] if s == 'star' else 
            self.colors['P2_line'] if s == 'square' else 
            self.colors['x_line'] 
            for s in symbols
        ]
        
        # Set border thickness for 'x' markers to be thicker than others
        line_widths = [4 if s == 'x' else 3 for s in symbols]  
        
        # Add markers to the figure
        fig.add_scatter(
            x=x, y=y, mode='markers',
            marker=dict(
                size=40,
                symbol=symbols,
                color=colors,
                line=dict(
                    width=line_widths,
                    color=line_colors
                )
            )
        )
        
        # Configure the layout of the figure (board appearance)
        fig.update_layout(
            width=700, height=700,
            paper_bgcolor=self.colors['background'],
            plot_bgcolor=self.colors['background'],
            font=dict(color=self.colors['text'], size=14),
            margin=dict(l=20, r=20, t=20, b=20)
        )
        
        # Configure the grid for the board (axis appearance and grid lines)
        axis_config = dict(
            range=[0, self.N],
            gridcolor=self.colors['grid'],
            linecolor=self.colors['grid'],
            tickvals=list(range(self.N + 1)),
            tickfont=dict(size=12)
        )
        
        fig.update_xaxes(**axis_config)
        fig.update_yaxes(**axis_config, autorange='reversed') 
        
        # Attach a click event handler to handle user interaction on the board
        fig.data[0].on_click(self.handle_human_turn)
        
        return fig


    def handle_restart_click(self, bt_restart):
        """Reset the game and initiate a new turn."""
        self.restart()
        self.start_player_turn()

    def map_state_to_visual_symbols(self):
        """Map the game state to corresponding visual symbols."""
        return [self.symbols_fig[v] for row in self.state for v in row]

    def start_player_turn(self):
        """Begin a player's turn and handle AI move if applicable."""
        player_types = {self.player_1['type'], self.player_2['type']}
        
        if 'human' not in player_types:
            while self.winner == 'keep_playing':
                self.handle_ai_turn() # AI makes a move until the game ends
        elif self.next_to_play['type'] == 'AI':
            self.handle_ai_turn() # AI makes its move if it's their turn

    def wait_for_player_move(self):
        """Waits for the player's move."""
        try:
            script = self.next_to_play['script']
            state_copy = deepcopy(self.state)
            move = script.get_move_official(None, state_copy, self.next_to_play['symbol'])
            return move or 'не одговори' # Return the move or default response
        except Exception as e:
            print(f"Грешка при преземање на потег: {e}")
            return 'не одговори'  # Return default response in case of error


    def handle_ai_turn(self):
        """Process the AI's move and update game state."""
        self.dashboard.value += ' пресметува🤔💭'  # Indicate AI is processing
        name = self.next_to_play["name"]
        move = self.wait_for_player_move()  # Get the move from the AI
        qx, qy = self.find_amazon(self.state, self.next_to_play['symbol'])
        valid, reason = self.validate_move_response(move)

        if valid:
            self.perform_move(name, move, qx, qy)  # Execute the valid AI move
        else:
            self.handle_invalid_move(name, qx, qy, reason)  

        self.update_game_state_visuals()  # Update the game state after the move
        self.complete_player_turn()  # Mark the turn as complete

    
    def perform_move(self, name, move, qx, qy):
        """Carry out a valid AI move and update the game state."""
        self.print_to_dash(f'{name}: {move}.') # Display AI's move on dashboard
        mx, my, sx, sy = move
        self.state[my][mx] = self.state[qy][qx] # Move the piece on the board
        self.state[qy][qx] = '·' # Clear the previous position
        self.update_game_state_visuals() # Update the state after the move
        self.state[sy][sx] = 'x' # Mark the destination with an 'x' 
    
    def handle_invalid_move(self, name, qx, qy, reason):
        """Handle an invalid move by selecting a random valid move."""
        msg = f'{name} - Невалиден чекор: {reason}. Ќе се избере рандом чекор.'
        self.print_to_dash(msg)
        
        mx, my = random.choice(list(self.valid_moves(self.state, qx, qy)))
        self.state[my][mx] = self.state[qy][qx]
        self.state[qy][qx] = '·'
        self.update_game_state_visuals()
        
        sx, sy = random.choice(list(self.valid_moves(self.state, mx, my)))
        self.state[sy][sx] = 'x'


    def handle_human_turn(self, trace, points, selector):
        """Process a move made by the human player and update the game state."""
        
        # Convert clicked coordinates to grid indices
        x, y = int(points.xs[0] - 0.5), int(points.ys[0] - 0.5)
        
        # Ensure coordinates are within grid boundaries
        x = max(0, min(x, self.N - 1))
        y = max(0, min(y, self.N - 1))
    
        if self.winner != 'keep_playing' or self.next_to_play['type'] != 'human':
            return # Exit if the game has ended or it's not the human's turn
        
        if self.next_human_move == 'amazon_moves':
             # If the amazon is moving, check if the move is valid
            amazon_x, amazon_y = self.find_amazon(self.state, self.next_to_play['symbol'])
            if (x, y) in self.valid_moves(self.state, amazon_x, amazon_y):
                # Perform the amazon move
                self.state[y][x], self.state[amazon_y][amazon_x] = self.state[amazon_y][amazon_x], '·'
                self.update_game_state_visuals()
                self.next_human_move = 'amazon_shoots'
                self.dashboard.value += ' Стрелај!'
        
        elif self.next_human_move == 'amazon_shoots':
            # If the amazon is shooting, check if the shot is valid
            amazon_x, amazon_y = self.find_amazon(self.state, self.next_to_play['symbol'])
            if (x, y) in self.valid_moves(self.state, amazon_x, amazon_y):
                # Perform the shooting action
                self.state[y][x] = 'x'
                self.update_game_state_visuals()
                self.next_human_move = 'amazon_moves'
                self.complete_player_turn()
                
                # If it's the AI's turn, trigger the AI move
                if self.next_to_play['type'] == 'AI' and self.winner == 'keep_playing':
                    self.handle_ai_turn()

    def validate_move_response(self, move):
        """Validate a player's move and return appropriate error messages if invalid."""
        error_messages = {
            'no_response': 'Играчот не одговори на време.',
            'none_response': "Одговорот на играчот е None.",
            'invalid_type': "Одговорот на играчот не е торка или листа.",
            'invalid_length': "Одговорот на играчот не е торка со должина 4.",
            'non_integer': "Одговорот на играчот содржи вредности кои не се цели броеви.",
            'out_of_range': "Одговорот на играчот содржи вредности надвор од дозволениот опсег.",
            'invalid_amazon_move': 'Амазонката се обидува да се премести на зафатено место {}.',
            'invalid_shot': 'Амазонката се обидува да пука на зафатено место {}.'
        }
        
        # Check for various types of invalid moves
        if move == 'did not respond':
            return False, error_messages['no_response']
        if move is None:
            return False, error_messages['none_response']
        if not isinstance(move, (tuple, list)):
            return False, error_messages['invalid_type']
        if len(move) != 4:
            return False, error_messages['invalid_length']
        if any(not isinstance(v, int) for v in move):
            return False, error_messages['non_integer']
        if any(v < 0 or v >= self.N for v in move):
            return False, error_messages['out_of_range']
        
        mx, my, sx, sy = move
        state = deepcopy(self.state)
        qx, qy = self.find_amazon(state, self.next_to_play['symbol'])
        
        # Check if the amazon's move is valid
        if (mx, my) not in list(self.valid_moves(state, qx, qy)):
            return False, error_messages['invalid_amazon_move'].format((mx, my))
        
        # Move the amazon to the new position
        state[my][mx], state[qy][qx] = state[qy][qx], '·'
        
        # Check if the shot is valid
        if (sx, sy) not in list(self.valid_moves(state, mx, my)):
            return False, error_messages['invalid_shot'].format((sx, sy))
        
        return True, '' # Move is valid

    def update_game_state_visuals(self):
        """Update the visual representation of the game state after a change."""
        def get_fill_color(symbol):
            return (
                self.colors['P1_fill'] if symbol == 'star' else
                self.colors['P2_fill'] if symbol == 'square' else
                self.colors['x_fill'] if symbol == 'x' else
                'rgba(0,0,0,0)' # Default transparent color 
            )
    
        def get_line_color(symbol):
            return (
                self.colors['P1_line'] if symbol == 'star' else
                self.colors['P2_line'] if symbol == 'square' else
                self.colors['x_line'] if symbol == 'x' else
                self.colors['grid'] # Default grid line color 
            )
    
        symbols = [self.symbols_fig[v] for row in self.state for v in row]  
        colors = [get_fill_color(s) for s in symbols] 
        line_colors = [get_line_color(s) for s in symbols] 
        
        marker = self.fig.data[0].marker
        marker.symbol = symbols
        marker.color = colors
        marker.line.color = line_colors
        
        if self.next_to_play['type'] == 'AI':
            sleep(0.5)

    def switch_to_next_player(self):
        """Switch the turn to the other player."""
        if self.next_to_play == self.player_2:
            return self.player_1 # Switch to player 1
        return self.player_2 # Switch to player 2

    def complete_player_turn(self):
        """Handle the transition after a player's turn ends."""
        self.next_to_play = self.switch_to_next_player() # Set the next player to play
        self.winner = self.evaluate_victory(self.state, self.next_to_play['symbol']) # Check for a winner
        
        if self.winner != 'keep_playing':
            self.announce_winner() # Declare the winner if there's one
            return
        
        self.display_turn_message()  # Update the message for the new turn

    def announce_winner(self):
        """Announce the winner and update the game score."""
        winner = self.switch_to_next_player()  # The player who just finished their turn is the winner
        self.update_score('Победник е', winner)  # Update the score with the winner's name

    def display_turn_message(self):
        """Display a message indicating whose turn it is."""
        self.update_score('На ред е', self.next_to_play)
        
        if self.next_to_play['type'] == 'human':
            self.dashboard.value += ' Помести ја амазонката!'

    def update_score(self, message, player):
        """Update the game dashboard with the current game status and player information."""
        player_data = f"{player['name']} ({player['symbol']})"
        self.dashboard.value = f"<span style='font-size: 18px;'>{message} <b>{player_data}</b></span>"

    def print_to_dash(self, msg):
        with self.output:
            print(msg)

    def find_amazon(self, state, symbol):
        """Find and return the coordinates of the specified amazon on the board."""
        for y in range(len(state)):
            for x in range(len(state[y])):
                if state[y][x] == symbol:
                    return x, y
        return None


    def valid_moves(self, state, x, y):
        """Yield all possible moves from the given position based on board constraints."""
        directions = [
            (0, 1), (0, -1), (1, 0), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)
        ]
        board_size = len(state)

        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy

            while 0 <= new_x < board_size and 0 <= new_y < board_size:
                if state[new_y][new_x] == '·':
                    yield new_x, new_y
                else:
                    break

                new_x += dx
                new_y += dy


    def get_opponent_amazon(self, amazon_symbol):
        """Get the opponent's amazon symbol."""
        return 'P2' if amazon_symbol == 'P1' else 'P1'

    def evaluate_victory(self, state, amazon_symbol):
        """Check if the game has ended and return the winner if any."""
        amazon_x, amazon_y = self.find_amazon(state, amazon_symbol)
        if not list(self.valid_moves(state, amazon_x, amazon_y)):
            return self.get_opponent_amazon(amazon_symbol)
        return 'keep_playing'

In [8]:
class StrategyAI:
    def __init__(self, algorithm='minimax', max_depth=2):
        self.algorithm = algorithm
        self.max_depth = max_depth
        self.scores = {'S': 1, 'P': -1}
        self.start_time = 0
        self.decision_times = [] 

        
    def get_move_official(self, queue, state, symbol):
        try:
            start_time = time.time()  # Start time tracking
            best_move = self.make_best_move(state, symbol)
            end_time = time.time()  # End time tracking
            
            decision_time = end_time - start_time
            self.decision_times.append(decision_time)  # Store decision time

            # Calculate the average decision time dynamically
            avg_time = sum(self.decision_times) / len(self.decision_times)
            print(f"{self.algorithm} време на одлука: {decision_time:.4f} секунди (Просек: {avg_time:.4f} секунди)")

            return best_move
        except Exception as e:
            print(f"Грешка при пресметка на AI: {e}")
            return self.make_random_move(state, symbol)

    def print_final_average(self):
        """Print the final average decision time when the game ends."""
        if self.decision_times:
            avg_time = sum(self.decision_times) / len(self.decision_times)
            print(f"\nПросечно време на одлука на {self.algorithm}: {avg_time:.4f} секунди")


    def make_random_move(self, state, symbol):
        """Make a random valid move for the given symbol."""
        try:
            # Find the position of the amazon
            amazon_x, amazon_y = self.find_amazon(state, symbol)

            # Get all possible moves for the amazon
            possible_moves = list(self.valid_moves(state, amazon_x, amazon_y))
            if not possible_moves:
                return None

            # Select a random move
            move_x, move_y = random.choice(possible_moves)

            # Create a temporary state to simulate the move
            temp_state = [list(row) for row in state]
            temp_state[move_y][move_x] = symbol  # Place the amazon in the new position
            temp_state[amazon_y][amazon_x] = '·'   # Clear the old position

            # Get all possible shots from the new position
            possible_shots = list(self.valid_moves(temp_state, move_x, move_y))
            if not possible_shots:
                return None

            # Select a random shot
            shot_x, shot_y = random.choice(possible_shots)

            return (move_x, move_y, shot_x, shot_y)  

        except Exception as error:
            print(f"Грешка при правење на рандом чекор: {error}")
            return None


    def make_best_move(self, state, symbol):
        """Compute the best move based on selected strategy"""
        self.start_time = time.time()
        current_depth = 1
        best_move = None
    
        while current_depth <= self.max_depth and (time.time() - self.start_time) < 2.5:
            try:
                move = None
    
                if self.algorithm == 'minimax':
                    _, move = self.minimax(state, 'MAX', symbol, depth=0, max_depth=current_depth)
                elif self.algorithm == 'alphabeta':
                    _, move = self.alphabeta(state, 'MAX', symbol, float('-inf'), float('inf'), 
                                             depth=0, max_depth=current_depth)
                elif self.algorithm == 'expectimax':
                    _, move = self.expectimax(state, 'MAX', symbol, depth=0, max_depth=current_depth)
    
                if move is not None:
                    best_move = move
    
                current_depth += 1
    
            except TimeoutError:
                break
    
        return best_move if best_move is not None else self.make_random_move(state, symbol)


    def minimax(self, state, player, symbol, depth=0, max_depth=2):
        """
        Minimax algorithm for decision-making in the game.
        """
        if depth == max_depth or self.check_for_winner(state, symbol) != 'keep_playing':
            return self.assess_state(state, symbol), None
    
        if player == 'MAX':  # Maximizing player
            best_value = float('-inf')
            best_move = None
            for next_state, move in self.expand_state(state, symbol):
                value, _ = self.minimax(next_state, 'MIN', self.get_opponent_amazon(symbol), depth + 1, max_depth)
                if value > best_value:
                    best_value, best_move = value, move
            return best_value, best_move
    
        else:  # Minimizing player
            best_value = float('inf')
            best_move = None
            for next_state, move in self.expand_state(state, symbol):
                value, _ = self.minimax(next_state, 'MAX', self.get_opponent_amazon(symbol), depth + 1, max_depth)
                if value < best_value:
                    best_value, best_move = value, move
            return best_value, best_move

    def alphabeta(self, state, player, symbol, alpha, beta, depth=0, max_depth=3):
        """
        Alpha-Beta Pruning implementation for optimized minimax decision-making.
        """
        if depth == max_depth or self.check_for_winner(state, symbol) != 'keep_playing':
            return self.assess_state(state, symbol), None
    
        if player == 'MAX':
            best_value = float('-inf')
            best_move = None
            for next_state, move in self.expand_state(state, symbol):
                value, _ = self.alphabeta(next_state, 'MIN', self.get_opponent_amazon(symbol), alpha, beta, depth + 1, max_depth)
                if value > best_value:
                    best_value, best_move = value, move
                alpha = max(alpha, best_value)
                if beta <= alpha:
                    break  # Prune the search tree
            return best_value, best_move
    
        else:
            best_value = float('inf')
            best_move = None
            for next_state, move in self.expand_state(state, symbol):
                value, _ = self.alphabeta(next_state, 'MAX', self.get_opponent_amazon(symbol), alpha, beta, depth + 1, max_depth)
                if value < best_value:
                    best_value, best_move = value, move
                beta = min(beta, best_value)
                if beta <= alpha:
                    break  # Prune the search tree
            return best_value, best_move

        
    def expectimax(self, state, player, symbol, depth=0, max_depth=2):
        """
        Expectimax algorithm for stochastic decision-making.
        """
        if depth == max_depth or self.check_for_winner(state, symbol) != 'keep_playing':
            return self.assess_state(state, symbol), None
    
        if player == 'MAX':  # Maximizing player
            return self.maximizer(self.expand_state(state, symbol), symbol, depth, max_depth)
    
        elif player == 'CHANCE':  # Chance player 
            return self.expectation(self.expand_state(state, symbol), symbol, depth, max_depth)
    
    def get_opponent_amazon(self, amazon_symbol):
        """Get the opponent's amazon symbol."""
        return 'P2' if amazon_symbol == 'P1' else 'P1'

    def maximizer(self, moves, symbol, depth, max_depth):
        """Helper function for MAX player decision-making with randomness."""
        best_value = float('-inf')
        best_moves = []
        
        for next_state, move in moves:
            other_symbol = 'P2' if symbol == 'P1' else 'P1'
            value, _ = self.expectimax(next_state, 'CHANCE', other_symbol, depth + 1, max_depth)
            
            # Add randomness to the evaluation
            value += random.uniform(-0.1, 0.1)
            
            if value > best_value:
                best_value = value
                best_moves = [move]
            elif value == best_value:
                best_moves.append(move)
        
        best_move = random.choice(best_moves) if best_moves else None
        return best_value, best_move

    def expectation(self, moves, symbol, depth, max_depth):
        """Helper function for CHANCE player decision-making with randomness."""
        total_value = 0
        num_moves = len(moves)
        
        for next_state, _ in moves:
            value, _ = self.expectimax(next_state, 'MAX', 'P2' if symbol == 'P1' else 'P1', depth + 1, max_depth)
            
            # Add randomness to each child evaluation
            value += random.uniform(-0.05, 0.05)
            
            total_value += value
        
        # Add a small random factor to the final expectation
        average_value = (total_value / num_moves) if num_moves > 0 else 0
        final_value = average_value + random.uniform(-0.02, 0.02)
        
        return final_value, None

    def check_for_winner(self, state, symbol):
        """Check if the game is over by determining if the amazon has no possible moves."""
        amazon_x, amazon_y = self.find_amazon(state, symbol)
        has_moves = any(self.valid_moves(state, amazon_x, amazon_y))
        
        if not has_moves:
            return 'P2' if symbol == 'P1' else 'P1'
        
        return 'keep_playing'

    def find_amazon(self, state, symbol):
        """Find and return the coordinates of the specified amazon on the board."""
        board_size = len(state)
        
        for y in range(board_size):
            for x in range(board_size):
                if state[y][x] == symbol:
                    return x, y
        
        return None

    def valid_moves(self, state, x, y):
        """Yield all possible moves from the given position based on board constraints."""
        directions = [
            (0, 1), (0, -1), (1, 0), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)
        ]
        board_size = len(state)
        
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            
            while 0 <= new_x < board_size and 0 <= new_y < board_size:
                if state[new_y][new_x] == '·':
                    yield new_x, new_y
                else:
                    break
                
                new_x += dx
                new_y += dy

    def expand_state(self, state, symbol):
        """Generate all possible next states from the current state."""
        qx, qy = self.find_amazon(state, symbol)  # Find the amazon's position
        expanded_states = []

        for move_x, move_y in self.valid_moves(state, qx, qy):
            # Create a copy of the state after the amazon's move
            next_state = [list(row) for row in state]
            next_state[move_y][move_x] = symbol
            next_state[qy][qx] = '·'

            for shot_x, shot_y in self.valid_moves(next_state, move_x, move_y):
                # Create a deep copy of the state after the arrow shot
                final_state = deepcopy(next_state)
                final_state[shot_y][shot_x] = 'x'

                # Convert the state into an immutable tuple representation
                final_state = tuple(tuple(row) for row in final_state)
                expanded_states.append((final_state, (move_x, move_y, shot_x, shot_y)))

        # Sort states before returning (best moves first)
        expanded_states.sort(key=lambda move: self.assess_state(move[0], symbol), reverse=True)
        return expanded_states


    def assess_state(self, state, symbol):
        """Evaluate the current state for the given symbol with improved mobility scoring."""

        # Determine positions of the player and opponent
        my_pos = self.find_amazon(state, symbol)
        opponent_symbol = 'P2' if symbol == 'P1' else 'P1'
        opp_pos = self.find_amazon(state, opponent_symbol)

        # If either player is missing, return neutral evaluation
        if not my_pos or not opp_pos:
            return 0

        # Calculate mobility (number of possible moves)
        my_moves = len(list(self.valid_moves(state, *my_pos)))
        opp_moves = len(list(self.valid_moves(state, *opp_pos)))

        # Calculate distance from the center
        center = len(state) // 2
        my_center_dist = sum(abs(coord - center) for coord in my_pos)
        opp_center_dist = sum(abs(coord - center) for coord in opp_pos)

        # Compute scores based on mobility and position
        mobility_score = (my_moves - opp_moves) / max(1, my_moves + opp_moves)  # Prevent division by zero
        position_score = (opp_center_dist - my_center_dist) / (len(state) * 2)

        # Adjust weights
        base_score = 0.75 * mobility_score + 0.25 * position_score  # Prioritize mobility

        # Reduce randomness to avoid instability
        if abs(mobility_score - position_score) < 0.1:  # If scores are close, allow slight randomness
            random_factor = random.uniform(0.99, 1.01)  # Tiny variation
        else:
            random_factor = 1  # No randomness if scores are very different

        return base_score * random_factor

In [9]:
class GameSetup:
    def __init__(self):
        self.initial_board = [
            ['·', '·', '·', '·', '·', '·'],
            ['·', '·', '·', '·', '·', 'P2'],
            ['·', '·', '·', '·', '·', '·'],
            ['·', '·', '·', '·', '·', '·'],
            ['P1', '·', '·', '·', '·', '·'],
            ['·', '·', '·', '·', '·', '·']
        ]
        self.ai_players = self.create_ai_players()
        self.players = self.setup_players()

    def manhattan_distance(self, pos1, pos2):
        """Calculate Manhattan distance between two positions."""
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

    def create_ai_players(self):
        """Initialize AI players with different algorithms."""
        return {
            'minimax': StrategyAI(algorithm='minimax', max_depth=2),
            'alphabeta': StrategyAI(algorithm='alphabeta', max_depth=2),
            'expectimax': StrategyAI(algorithm='expectimax', max_depth=2)
        }

    def setup_players(self):
        """Configure the player setup for AI vs AI."""
        return [
            {'name': 'MiniMaxAI', 'type': 'AI', 'script': self.ai_players['minimax']},
            {'name': 'ExpectiMaxAI', 'type': 'AI', 'script': self.ai_players['expectimax']}
            
        ]

    # def setup_players(self):
    #     """Configure the player setup for Human vs AI."""
    #     return [
    #         {'name': 'Human', 'type': 'human'},  # Player 1: Human
    #         {'name': 'MinimaxAI', 'type': 'AI', 'script': self.ai_players['minimax']}  # Player 2: AI
    #     ]

    def start_game(self):
        """Initialize and start the game."""
        game = Amazons(self.initial_board, *self.players, time_to_play=5)
        self.configure_game_display(game)
        game.start_player_turn()

    def configure_game_display(self, game):
        """Configure the game board visualization settings."""
        game.fig.update_layout(
            xaxis=dict(
                scaleanchor="y", scaleratio=1, showgrid=True, 
                gridwidth=2, gridcolor='black', 
                tickvals=list(range(len(game.state[0]) + 1))  
            ),
            yaxis=dict(
                scaleanchor="x", scaleratio=1, showgrid=True, 
                gridwidth=2, gridcolor='black', 
                tickvals=list(range(len(game.state) + 1))  
            )
        )

In [10]:
if __name__ == '__main__':
    game_setup = GameSetup()
    game_setup.start_game()
    

minimax време на одлука: 1.7171 секунди (Просек: 1.7171 секунди)
expectimax време на одлука: 1.0714 секунди (Просек: 1.0714 секунди)
minimax време на одлука: 1.2867 секунди (Просек: 1.5019 секунди)
expectimax време на одлука: 0.8713 секунди (Просек: 0.9713 секунди)
minimax време на одлука: 0.5245 секунди (Просек: 1.1761 секунди)
expectimax време на одлука: 0.1441 секунди (Просек: 0.6956 секунди)
minimax време на одлука: 0.2594 секунди (Просек: 0.9469 секунди)
expectimax време на одлука: 0.3135 секунди (Просек: 0.6001 секунди)
minimax време на одлука: 0.2153 секунди (Просек: 0.8006 секунди)
expectimax време на одлука: 0.1025 секунди (Просек: 0.5006 секунди)
minimax време на одлука: 0.1435 секунди (Просек: 0.6911 секунди)
expectimax време на одлука: 0.1070 секунди (Просек: 0.4350 секунди)
minimax време на одлука: 0.0920 секунди (Просек: 0.6055 секунди)
expectimax време на одлука: 0.0704 секунди (Просек: 0.3829 секунди)
minimax време на одлука: 0.0651 секунди (Просек: 0.5380 секунди)
expe